In [2]:
!pip install azure-search-documents
!pip install openai
!pip install pandas



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [3]:
import os

from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.models import VectorizedQuery
from azure.search.documents.indexes.models import (
        SearchIndex,
        SearchField,
        SearchFieldDataType,
        SimpleField,
        SearchableField,
        VectorSearch,
        VectorSearchProfile,
        HnswAlgorithmConfiguration,
    )
import pandas as pd
from azure.core.exceptions import ResourceNotFoundError





In [4]:
ord("A") - ord("a")

-32

In [63]:
import time
from openai import AzureOpenAI

client = AzureOpenAI(
    api_key="",  # replace with env var or secret manager in real use
    api_version="2024-02-15-preview",
    azure_endpoint="https://genaitcgazuregpt.openai.azure.com",
)

def get_embeddings(text: str):
    """Get embeddings for text using Azure OpenAI"""
    if not text or text.strip() == "":
        return [0.0] * 1536

    try:
        response = client.embeddings.create(
            input=[text],
            model="text-embedding-ada-002"
        )
        return response.data[0].embedding
    except Exception as e:
        print(f"Error getting embeddings for text '{text[:50]}...': {str(e)}")
        return [0.0] * 1536

# Test
start = time.time()
print(get_embeddings("hi"))
end = time.time()
print("Time taken:", end - start)


Error getting embeddings for text 'hi...': Error code: 401 - {'error': {'code': '401', 'message': 'Access denied due to invalid subscription key or wrong API endpoint. Make sure to provide a valid key for an active subscription and use a correct regional API endpoint for your resource.'}}
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

In [54]:
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import (
    SearchIndex, SimpleField, SearchableField, SearchFieldDataType,
    SearchField, VectorSearch, VectorSearchProfile, HnswAlgorithmConfiguration
)

def create_index():
    index_client = SearchIndexClient(endpoint=service_endpoint, credential=credential)

    fields = [
        SimpleField(name="id", type=SearchFieldDataType.String, key=True),
        SearchableField(name="content", type=SearchFieldDataType.String),
        SearchField(
            name="embedding",
            type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
            searchable=True,
            vector_search_dimensions=1536,
            vector_search_profile_name="my-vector-profile"
        )
    ]

    vector_search = VectorSearch(
        profiles=[
            VectorSearchProfile(
                name="my-vector-profile",
                algorithm_configuration_name="my-hnsw"
            )
        ],
        algorithms=[
            HnswAlgorithmConfiguration(name="my-hnsw")
        ]
    )

    index = SearchIndex(name=index_name, fields=fields, vector_search=vector_search)

    try:
        if index_name in [idx.name for idx in index_client.list_indexes()]:
            index_client.delete_index(index_name)  # ⚠️ Delete existing schema
            print(f"Deleted old index: {index_name}")
        index_client.create_index(index)
        print(f"Created index: {index_name}")
    except Exception as e:
        print(f"Error creating index: {e}")


In [55]:
create_index()

Deleted old index: defects_new
Created index: defects_new


In [58]:
from azure.search.documents import SearchClient
import uuid

# Reuse existing credential and service_endpoint from earlier
search_client = SearchClient(endpoint=service_endpoint, index_name=index_name, credential=credential)

def add_documents_to_index(documents_text_list):
    docs = []
    for text in documents_text_list:
        doc_id = str(uuid.uuid4())
        embedding = get_embeddings(text)
        doc = {
            "id": doc_id,
            "content": text,
            "embedding": embedding
        }
        docs.append(doc)

    try:
        result = search_client.upload_documents(documents=docs)
        print("Upload results:", result)
    except Exception as e:
        print(f"Error uploading documents: {e}")


        
add_documents_to_index(["good", "bad", "ugly", "hi"])

Upload results: [<azure.search.documents._generated.models._models_py3.IndexingResult object at 0x7674081e2480>, <azure.search.documents._generated.models._models_py3.IndexingResult object at 0x7674081e1dc0>, <azure.search.documents._generated.models._models_py3.IndexingResult object at 0x7674081e3770>, <azure.search.documents._generated.models._models_py3.IndexingResult object at 0x7674081e03e0>]


In [59]:
from azure.search.documents import SearchClient

def extract_all_documents():
    search_client = SearchClient(endpoint=service_endpoint, index_name=index_name, credential=credential)
    
    try:
        results = search_client.search(search_text="*", top=1000)
        print("\n--- Retrieved Documents ---")

        all_docs = []
        for result in results:
            # If result is a dict (common), use dict access
            if isinstance(result, dict):
                doc = {
                    "id": result.get("id"),
                    "content": result.get("content"),
                    "score": result.get("@search.score")
                }
            else:
                # Fallback if result is a SearchResult object
                doc = {
                    "id": getattr(result, "id", None),
                    "content": getattr(result, "content", None),
                    "score": result.get("@search.score") if "@search.score" in result else None
                }
            print(doc)
            all_docs.append(doc)
        return all_docs

    except Exception as e:
        print(f"Error fetching documents: {e}")
        return []

# Call the function
extract_all_documents()



--- Retrieved Documents ---
{'id': '9c03e519-a00a-4f2c-98ba-df42ed078881', 'content': 'bad', 'score': 1.0}
{'id': '1794ff3b-3e0e-444a-b04c-03abfdf78d4c', 'content': 'ugly', 'score': 1.0}
{'id': '28f2e087-19b3-4562-9626-93669cd42dfa', 'content': 'hi', 'score': 1.0}
{'id': 'f696a3f8-5073-4e9e-8559-e555c828e38b', 'content': 'good', 'score': 1.0}
{'id': '05b9dffb-8d50-4ca5-8ac9-48ab351ccb1f', 'content': 'hi', 'score': 1.0}


[{'id': '9c03e519-a00a-4f2c-98ba-df42ed078881',
  'content': 'bad',
  'score': 1.0},
 {'id': '1794ff3b-3e0e-444a-b04c-03abfdf78d4c',
  'content': 'ugly',
  'score': 1.0},
 {'id': '28f2e087-19b3-4562-9626-93669cd42dfa', 'content': 'hi', 'score': 1.0},
 {'id': 'f696a3f8-5073-4e9e-8559-e555c828e38b',
  'content': 'good',
  'score': 1.0},
 {'id': '05b9dffb-8d50-4ca5-8ac9-48ab351ccb1f', 'content': 'hi', 'score': 1.0}]

In [60]:
from azure.search.documents import SearchClient
from azure.search.documents.models import VectorizedQuery

def search_by_embedding(query_text, k=3):
    """Search documents similar to the query_text using vector search."""
    search_client = SearchClient(endpoint=service_endpoint, index_name=index_name, credential=credential)
    
    try:
        embedding = get_embeddings(query_text)
        vector_query = VectorizedQuery(
            vector=embedding,
            k_nearest_neighbors=k,
            fields="embedding"
        )
        
        results = search_client.search(
            vector_queries=[vector_query],
            top=k
        )

        print(f"\n--- Search Results for: '{query_text}' ---")
        all_results = []
        for result in results:
            doc = {
                "id": result["id"] if "id" in result else None,
                "content": result["content"] if "content" in result else None,
                "score": result["@search.score"] if "@search.score" in result else None
            }
            print(doc)
            all_results.append(doc)

        return all_results

    except Exception as e:
        print(f"Search error: {e}")
        return []

# Example usage
search_by_embedding("hi")



--- Search Results for: 'hi' ---
{'id': '28f2e087-19b3-4562-9626-93669cd42dfa', 'content': 'hi', 'score': 1.0000001}
{'id': '05b9dffb-8d50-4ca5-8ac9-48ab351ccb1f', 'content': 'hi', 'score': 1.0000001}
{'id': 'f696a3f8-5073-4e9e-8559-e555c828e38b', 'content': 'good', 'score': 0.8760326}


[{'id': '28f2e087-19b3-4562-9626-93669cd42dfa',
  'content': 'hi',
  'score': 1.0000001},
 {'id': '05b9dffb-8d50-4ca5-8ac9-48ab351ccb1f',
  'content': 'hi',
  'score': 1.0000001},
 {'id': 'f696a3f8-5073-4e9e-8559-e555c828e38b',
  'content': 'good',
  'score': 0.8760326}]